In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
import datetime as dt
from scipy import stats

In [2]:
pd.set_option('display.max.columns', None)
pd.set_option('display.float_format', '{:.1f}'.format)

In [3]:
customers = pd.read_csv('olist_customers_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
order_payments = pd.read_csv('olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
category = pd.read_csv('category_name.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
geo = pd.read_csv('olist_geolocation_dataset.csv')

In [4]:
# Объединяем данные в один датафрейм:
join_1 = order_items.merge(orders, on='order_id', how='left')
join_2 = join_1.merge(products, on='product_id', how='left')
join_3 = join_2.merge(customers, on='customer_id', how='left')
join_4 = join_3.merge(order_payments, on='order_id', how='left')
df = join_4.merge(order_reviews, on='order_id', how='left')

In [5]:
# Изменим форматирование дат на datetime:
columns = ['shipping_limit_date', 'order_purchase_timestamp', 'order_approved_at',\
                'order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date',\
               'review_creation_date', 'review_answer_timestamp']
for col in columns:
    df[col] = pd.to_datetime(df[col])

In [6]:
# Преобразуем данные, добавив в т.ч. несколько столбцов:
df = df.rename({'order_purchase_timestamp': 'date_purchase', 'review_answer_timestamp': 'review_answer'}, axis=1)

In [7]:
# Год
df['year'] = df.date_purchase.dt.year
# Дату, без года
df['day_purchase'] = df.date_purchase.dt.date
# День недели 
df['weekday'] = df.date_purchase.dt.dayofweek # где 0 - пн., 6 - вс. 
# Пару год-месяц
df['year_month'] = df['day_purchase'].apply(lambda x:x.strftime('%Y_%m'))

In [8]:
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,date_purchase,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer,year,day_purchase,weekday,year_month
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.3,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,1.0,credit_card,2.0,72.2,97ca439bc427b48bc1cd7177abe71365,5,NaN,"Perfeito, produto entregue antes do combinado.",2017-09-21,2017-09-22 10:57:03,2017,2017-09-13,2,2017_09
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.9,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1.0,credit_card,3.0,259.8,7b07bacd811c4117b742569b04ce3580,4,NaN,NaN,2017-05-13,2017-05-15 11:34:13,2017,2017-04-26,2,2017_04
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.9,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,1.0,credit_card,5.0,216.9,0c5b33dea94867d1ac402749e5438e8b,5,NaN,Chegou antes do prazo previsto e o produto sur...,2018-01-23,2018-01-23 16:06:31,2018,2018-01-14,6,2018_01
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,13.0,12.8,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,1.0,credit_card,2.0,25.8,f4028d019cb58564807486a6aaf33817,4,NaN,NaN,2018-08-15,2018-08-15 16:39:01,2018,2018-08-08,2,2018_08
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9,18.1,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,1.0,credit_card,3.0,218.0,940144190dcba6351888cafa43f3a3a5,5,NaN,Gostei pois veio no prazo determinado .,2017-03-02,2017-03-03 10:54:59,2017,2017-02-04,5,2017_02


In [11]:
# Отберем актуальные заказы:
actual_orders = df[~df['order_status'].isin(['canceled', 'unavailable'])]
actual_orders.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,date_purchase,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer,year,day_purchase,weekday,year_month
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.3,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,1.0,credit_card,2.0,72.2,97ca439bc427b48bc1cd7177abe71365,5,NaN,"Perfeito, produto entregue antes do combinado.",2017-09-21,2017-09-22 10:57:03,2017,2017-09-13,2,2017_09
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.9,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1.0,credit_card,3.0,259.8,7b07bacd811c4117b742569b04ce3580,4,NaN,NaN,2017-05-13,2017-05-15 11:34:13,2017,2017-04-26,2,2017_04
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.9,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,1.0,credit_card,5.0,216.9,0c5b33dea94867d1ac402749e5438e8b,5,NaN,Chegou antes do prazo previsto e o produto sur...,2018-01-23,2018-01-23 16:06:31,2018,2018-01-14,6,2018_01
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,13.0,12.8,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,1.0,credit_card,2.0,25.8,f4028d019cb58564807486a6aaf33817,4,NaN,NaN,2018-08-15,2018-08-15 16:39:01,2018,2018-08-08,2,2018_08
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9,18.1,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,1.0,credit_card,3.0,218.0,940144190dcba6351888cafa43f3a3a5,5,NaN,Gostei pois veio no prazo determinado .,2017-03-02,2017-03-03 10:54:59,2017,2017-02-04,5,2017_02


In [15]:
# День для расчета давности последней покупки
rday = actual_orders['day_purchase'].max() + dt.timedelta(1)
rday

In [175]:
# Заказом считаем заказ с уникальным order_id
# Другой подход для расчета frequency - учитывать транзакции
rfm = actual_orders.groupby('customer_unique_id').\
    agg({'day_purchase': lambda x: (rday - x.max()).days,'order_id': lambda x: x.nunique(), 'payment_value': 'sum'}).reset_index()

In [176]:
rfm.rename(columns = {'day_purchase' : 'recency', 
                      'order_id' : 'frequency', 
                      'payment_value' : 'monetary'}, inplace = True)

In [85]:
# Посмотрим данные по одному из покупателей
rfm.loc[(rfm['customer_unique_id'] == '9cc5a07f169a1606fd347a56683e6ea6')] # покупатель с 4-мя заказами
# rfm.loc[(rfm['customer_unique_id'] == '000fbf0473c10fc1ab6f8d2d286ce20c')] # другой покупатель с 4-мя заказами
# rfm.loc[(rfm['customer_unique_id'] == '8d50f5eadf50201ccdcedfb9e2ac8455')] # ведущий (по числу заказов) покупатель
# rfm.loc[(rfm['customer_unique_id'] == '00fd4a627a3b93cff0729f0b3554640d')] # покупатель с 3-мя заказами

,customer_unique_id,recency,frequency,monetary
58244,9cc5a07f169a1606fd347a56683e6ea6,13,4,694.7


In [86]:
# В исходном массиве записи по нему:
actual_orders.loc[(actual_orders['customer_unique_id'] == '9cc5a07f169a1606fd347a56683e6ea6')]
# actual_orders.loc[(actual_orders['customer_unique_id'] == '8d50f5eadf50201ccdcedfb9e2ac8455')] # ведущий (по числу заказов) покупатель
# actual_orders.loc[(actual_orders['customer_unique_id'] == '00fd4a627a3b93cff0729f0b3554640d')] # покупатель с 3-мя заказами
# actual_orders.loc[(actual_orders['customer_unique_id'] == '000fbf0473c10fc1ab6f8d2d286ce20c')] # покупатель с 4-мя заказами

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,date_purchase,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer,year,day_purchase,weekday,year_month
50267,6c6803a80161b45ad151601b21ba647f,1,9bb8ca338e5588c361e34eae02e8fad6,620c87c171fb2a6dd6e8bb4dec959fc6,2017-12-22 02:17:32,69.9,12.5,ca0c06a18dd6faf5e07fbb8da4f32257,delivered,2017-12-13 21:34:30,2017-12-15 02:17:32,2017-12-15 17:33:06,2017-12-18 20:59:05,2018-01-03,beleza_saude,37.0,314.0,1.0,431.0,19.0,17.0,15.0,9cc5a07f169a1606fd347a56683e6ea6,21931,rio de janeiro,RJ,1.0,boleto,1.0,82.4,5fbdc041e0938773613c1a6fbbf7b62a,5,NaN,"Produto original, entrega rápida e bom preço!!!",2017-12-19,2017-12-20 14:20:52,2017,2017-12-13,2,2017_12
54623,75df014e897510b97edbb6261ef40f82,1,24c66f106f642621e524291a895c9032,620c87c171fb2a6dd6e8bb4dec959fc6,2017-12-26 02:17:31,159.9,13.1,61d1d67b171087d5bfcc5afaa0238c31,delivered,2017-12-13 21:34:31,2017-12-15 02:17:31,2017-12-15 17:33:58,2017-12-18 22:03:45,2018-01-04,beleza_saude,37.0,325.0,1.0,792.0,19.0,14.0,16.0,9cc5a07f169a1606fd347a56683e6ea6,21931,rio de janeiro,RJ,1.0,boleto,1.0,173.0,5fbdc041e0938773613c1a6fbbf7b62a,5,NaN,"Produto original, entrega rápida e bom preço!!!",2017-12-19,2017-12-20 14:20:52,2017,2017-12-13,2,2017_12
54624,75df014e897510b97edbb6261ef40f82,1,24c66f106f642621e524291a895c9032,620c87c171fb2a6dd6e8bb4dec959fc6,2017-12-26 02:17:31,159.9,13.1,61d1d67b171087d5bfcc5afaa0238c31,delivered,2017-12-13 21:34:31,2017-12-15 02:17:31,2017-12-15 17:33:58,2017-12-18 22:03:45,2018-01-04,beleza_saude,37.0,325.0,1.0,792.0,19.0,14.0,16.0,9cc5a07f169a1606fd347a56683e6ea6,21931,rio de janeiro,RJ,1.0,boleto,1.0,173.0,c141e288b3722caeb717911e52fee32f,5,NaN,"Produto original, bom preço e entrega rápida!!!",2017-12-19,2017-12-20 14:21:37,2017,2017-12-13,2,2017_12
61966,8623ddd82259f932337ad9f727cd25b5,1,e4eec4a0771ae2297acc444c8f559cfb,7b07b3c7487f0ea825fc6df75abd658b,2018-08-24 01:45:12,46.0,15.4,73c81eed8822aec98bc389ccce382294,delivered,2018-08-22 01:37:38,2018-08-22 01:45:12,2018-08-23 07:17:00,2018-08-27 20:32:18,2018-09-10,esporte_lazer,47.0,577.0,1.0,140.0,19.0,12.0,14.0,9cc5a07f169a1606fd347a56683e6ea6,21931,rio de janeiro,RJ,1.0,credit_card,1.0,61.4,77bfb2939f608efe2f9b25e7e4d70665,5,Sempre correta!,"Sempre recomendo a targaryen! Pelos produtos, ...",2018-08-28,2018-08-29 19:02:24,2018,2018-08-22,2,2018_08
66835,9105e733a6a8fcccf4fced822e1a67ae,1,f6584475ef4b8710cec93be723766326,620c87c171fb2a6dd6e8bb4dec959fc6,2018-01-04 02:17:25,89.9,12.6,874dfb94ae7ceab047226c11f81cba46,delivered,2017-12-13 21:34:31,2017-12-15 02:17:25,2017-12-15 17:33:01,2017-12-18 21:46:33,2018-01-12,beleza_saude,38.0,334.0,1.0,527.0,18.0,16.0,13.0,9cc5a07f169a1606fd347a56683e6ea6,21931,rio de janeiro,RJ,1.0,boleto,1.0,102.5,c141e288b3722caeb717911e52fee32f,5,NaN,"Produto original, bom preço e entrega rápida!!!",2017-12-19,2017-12-20 14:21:37,2017,2017-12-13,2,2017_12
66836,9105e733a6a8fcccf4fced822e1a67ae,1,f6584475ef4b8710cec93be723766326,620c87c171fb2a6dd6e8bb4dec959fc6,2018-01-04 02:17:25,89.9,12.6,874dfb94ae7ceab047226c11f81cba46,delivered,2017-12-13 21:34:31,2017-12-15 02:17:25,2017-12-15 17:33:01,2017-12-18 21:46:33,2018-01-12,beleza_saude,38.0,334.0,1.0,527.0,18.0,16.0,13.0,9cc5a07f169a1606fd347a56683e6ea6,21931,rio de janeiro,RJ,1.0,boleto,1.0,102.5,a6ea5f040752d085542387f63a521a6f,5,NaN,NaN,2017-12-19,2017-12-20 14:22:13,2017,2017-12-13,2,2017_12


In [87]:
rfm.head()
# rfm.describe()

,customer_unique_id,recency,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,117,1,141.9
1,0000b849f77a49e4a4ce2b2a4ca5be3f,120,1,27.2
2,0000f46a3911fa3c0805444483337064,543,1,86.2
3,0000f6ccb0745a6a4b88665a16c9f078,327,1,43.6
4,0004aac84e0df4da2b147fca70cf8255,294,1,196.9


In [177]:
# Для выделения сегментов используем квартили:
quantiles = rfm.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [144]:
quantiles

{'recency': {0.25: 120.0, 0.5: 225.0, 0.75: 353.0},
 'frequency': {0.25: 1.0, 0.5: 1.0, 0.75: 1.0},
 'monetary': {0.25: 63.99, 0.5: 113.29, 0.75: 203.08}}

In [178]:
def Rscore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMscore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [179]:
rfm['R'] = rfm['recency'].apply(Rscore, args=('recency', quantiles,))
rfm['F'] = rfm['frequency'].apply(FMscore, args=('frequency', quantiles,))
rfm['M'] = rfm['monetary'].apply(FMscore, args=('monetary', quantiles,))
rfm.head()

,customer_unique_id,recency,frequency,monetary,R,F,M
0,0000366f3b9a7992bf8c76cfdf3221e2,117,1,141.9,1,4,2
1,0000b849f77a49e4a4ce2b2a4ca5be3f,120,1,27.2,1,4,4
2,0000f46a3911fa3c0805444483337064,543,1,86.2,4,4,3
3,0000f6ccb0745a6a4b88665a16c9f078,327,1,43.6,3,4,4
4,0004aac84e0df4da2b147fca70cf8255,294,1,196.9,3,4,2


In [180]:
rfm['RFM'] = rfm.R.map(str) + rfm.F.map(str) + rfm.M.map(str)
rfm['RFMscore'] = rfm[['R', 'F', 'M']].sum(axis=1)
rfm.head()

,customer_unique_id,recency,frequency,monetary,R,F,M,RFM,RFMScore
0,0000366f3b9a7992bf8c76cfdf3221e2,117,1,141.9,1,4,2,142,7
1,0000b849f77a49e4a4ce2b2a4ca5be3f,120,1,27.2,1,4,4,144,9
2,0000f46a3911fa3c0805444483337064,543,1,86.2,4,4,3,443,11
3,0000f6ccb0745a6a4b88665a16c9f078,327,1,43.6,3,4,4,344,11
4,0004aac84e0df4da2b147fca70cf8255,294,1,196.9,3,4,2,342,9


In [114]:
rfm.describe()

,recency,frequency,monetary,R,F,M,RFMScore
count,94983.0,94983.0,94983.0,94983.0,94983.0,94983.0,94983.0
mean,244.3,1.0,213.4,2.5,3.9,2.5,8.9
std,153.0,0.2,643.8,1.1,0.5,1.1,1.7
min,1.0,1.0,0.0,1.0,1.0,1.0,3.0
25%,120.0,1.0,64.0,1.0,4.0,2.0,8.0
50%,225.0,1.0,113.3,2.0,4.0,3.0,9.0
75%,353.0,1.0,203.1,3.0,4.0,4.0,10.0
max,730.0,16.0,109312.6,4.0,4.0,4.0,12.0


In [181]:
# Получили 32 сегмента
# По частоте покупок получилось только 2 равные по количеству сегментов группы: 1 покупка и более 1 покупки
rfm['RFM'].value_counts()

443    6049
444    6028
343    5995
244    5972
142    5937
144    5896
242    5878
344    5804
243    5741
143    5673
342    5650
442    5573
341    5494
141    5480
441    5475
241    5451
111     527
211     502
311     445
411     367
112     193
212     187
312     176
412     155
113      78
313      75
213      72
413      62
414      18
214      12
114      12
314       6
Name: RFM, dtype: int64

In [113]:
# Топ покупателей
rfm[rfm['RFM'] == '111'].sort_values('monetary', ascending = False).head (10)

,customer_unique_id,recency,frequency,monetary,R,F,M,RFM,RFMScore
74401,c8460e4251689ba205045f3ea17884a1,27,4,27935.5,1,1,1,111,3
53545,906a8a4ec9f3d4c3e64fa6d1c4fe6009,46,2,11881.0,1,1,1,111,3
19026,33176de67c05eeed870fd49f234387a0,92,3,7006.2,1,1,1,111,3
11815,1f98d2384ff7a372e6a2d3bb75cbcd54,95,2,6907.1,1,1,1,111,3
1230,0341bbd5c969923a0f801b9e2d10a7b8,97,2,5999.0,1,1,1,111,3
36783,62b8761192c86f4ead06f4a433096a2b,62,2,3508.1,1,1,1,111,3
21394,397b44d5bb99eabf54ea9c2b41ebb905,79,4,3436.1,1,1,1,111,3
30625,525cda9909aa001ebed396f6e55eae01,28,2,3371.8,1,1,1,111,3
2992,07ff81a9676efa0286ddd3f09e7c0063,85,2,2905.9,1,1,1,111,3
59484,a017de101a9c4fab4995c1bc81e737ea,108,2,2803.2,1,1,1,111,3


In [182]:
# Можно сократить кол-во сегментов, повторив процедуру с использованием значений RFMScore
rfm4 = rfm

In [183]:
rfm.head()

,customer_unique_id,recency,frequency,monetary,R,F,M,RFM,RFMScore
0,0000366f3b9a7992bf8c76cfdf3221e2,117,1,141.9,1,4,2,142,7
1,0000b849f77a49e4a4ce2b2a4ca5be3f,120,1,27.2,1,4,4,144,9
2,0000f46a3911fa3c0805444483337064,543,1,86.2,4,4,3,443,11
3,0000f6ccb0745a6a4b88665a16c9f078,327,1,43.6,3,4,4,344,11
4,0004aac84e0df4da2b147fca70cf8255,294,1,196.9,3,4,2,342,9


In [195]:
# rfm.describe()

In [185]:
def RFMscore(x,p,d):
    if x <= d[p][0.25]:
        return 'platinum'
    elif x <= d[p][0.50]:
        return 'gold'
    elif x <= d[p][0.75]: 
        return 'silver'
    else:
        return 'bronze'

In [186]:
quantiles4 = rfm4[['RFMscore']].quantile(q=[0.25,0.5,0.75])
quantiles4 = quantiles4.to_dict()
quantiles4

{'RFMScore': {0.25: 8.0, 0.5: 9.0, 0.75: 10.0}}

In [187]:
rfm4['level'] = rfm4['RFMscore'].apply(RFMScore, args=('RFMscore', quantiles4,))

In [188]:
rfm4.head(15)

,customer_unique_id,recency,frequency,monetary,R,F,M,RFM,RFMScore,Level
0,0000366f3b9a7992bf8c76cfdf3221e2,117,1,141.9,1,4,2,142,7,platinum
1,0000b849f77a49e4a4ce2b2a4ca5be3f,120,1,27.2,1,4,4,144,9,gold
2,0000f46a3911fa3c0805444483337064,543,1,86.2,4,4,3,443,11,bronze
3,0000f6ccb0745a6a4b88665a16c9f078,327,1,43.6,3,4,4,344,11,bronze
4,0004aac84e0df4da2b147fca70cf8255,294,1,196.9,3,4,2,342,9,gold
5,0004bd2a26a76fe21f786e4fbd80607f,152,1,167.0,2,4,2,242,8,platinum
6,00050ab1314c0e55a6ca13cf7181fecf,137,1,35.4,2,4,4,244,10,silver
7,00053a61a98854899e70ed204dd4bafe,188,1,838.4,2,4,1,241,7,platinum
8,0005e1862207bf6ccc02e4228effd9a0,549,1,150.1,4,4,2,442,10,silver
9,0005ef4cd20d2893f0d9fbd94d3c0d97,176,1,129.8,2,4,2,242,8,platinum


In [190]:
rfm4.groupby(['level']).agg\
    ({'customer_unique_id': 'count', 'monetary': 'sum', 'frequency': 'mean', 'RFMscore': 'mean'}).reset_index()\
    .sort_values('monetary', ascending = False)

,Level,customer_unique_id,monetary,frequency,RFMScore
2,platinum,36782,12967319.7,1.1,7.1
1,gold,22780,4645485.4,1.0,9.0
3,silver,17540,1614735.2,1.0,10.0
0,bronze,17881,1037798.7,1.0,11.3


In [194]:
# Топ покупателей
# rfm4[rfm4['level'] == 'platinum'].sort_values('monetary', ascending = False).head (10)